In [1]:
# https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [6]:
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
           for i in range(len(test_sentence) - 2)]
word2index = {word: i for i, word in enumerate(set(test_sentence))}

In [9]:
class TrigramModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(TrigramModel, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 256)
        self.linear2 = nn.Linear(256, vocab_size)
        
    def forward(self, inputs):
        embeds = self.embedding(inputs).view(1, -1)
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim = 1)
        
        return log_probs
    
loss_f = nn.NLLLoss()
model = TrigramModel(len(word2index), 16, 2)
optimizer = optim.SGD(model.parameters(), lr = 0.001)

for epoch in range(10):
    for context, target in trigrams:
        context = torch.tensor([word2index[w] for w in context], dtype = torch.long)
        model.zero_grad()
        log_probs = model(context)
        loss = loss_f(log_probs, torch.tensor([word2index[target]], dtype = torch.long))
        loss.backward()
        optimizer.step()
        

In [11]:
data = [([test_sentence[i - 2], test_sentence[i - 1], 
          test_sentence[i + 1], test_sentence[i + 2]], 
         test_sentence[i]) for i in range(2, len(test_sentence) - 2)]

In [12]:
class CBoW(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(CBoW, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 256)
        self.linear2 = nn.Linear(256, vocab_size)
        
    def forward(self, inputs):
        embeds = self.embedding(inputs).view(1, -1)
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim = 1)
        
        return log_probs
    
loss_f = nn.NLLLoss()
model = CBoW(len(word2index), 16, 4)
optimizer = optim.SGD(model.parameters(), lr = 0.001)

for epoch in range(10):
    for context, target in data:
        context = torch.tensor([word2index[w] for w in context], dtype = torch.long)
        model.zero_grad()
        log_probs = model(context)
        loss = loss_f(log_probs, torch.tensor([word2index[target]], dtype = torch.long))
        loss.backward()
        optimizer.step()